**Dashboard Code**
----------------------------------------------------------------------------------------------

In [ ]:
# Install dependencies if needed

!pip install pandas
!pip install plotly
!pip install ipywidgets


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
""" DASHBOARD GRAPHS CODE """

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display


# Load & Prepare Dataset
def load_and_prepare(path):
    df = pd.read_csv(path)
    df.columns = [c.lower().strip() for c in df.columns]

    # remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains("unnamed")]

    # check required columns
    required = {"organisation", "location", "date", "mission_status", "price"}
    if not required.issubset(set(df.columns)):
        missing = required - set(df.columns)
        raise ValueError(f"Dataset missing required columns: {missing}")

    # Convert date
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Extract year
    df["launch_year"] = df["date"].dt.year

    # location → extract country
    df["location"] = df["location"].astype(str).apply(
        lambda x: x.split(",")[-1].strip() if "," in x else x.strip()
    )

    # clean price
    df["price"] = df["price"].astype(str).str.replace(",", "", regex=False)
    df["price"] = df["price"].replace(["", "-", " "], pd.NA)
    df["price"] = pd.to_numeric(df["price"], errors="coerce")

    # Convert mission_status into numeric success flag
    df["success_flag"] = df["mission_status"].apply(
        lambda x: 1 if str(x).lower().strip() == "success" else 0
    )

    return df



# Graph making
def template_plot(df, X, Y, chart_type="bar", agg_fn="count"):
    """
    Generic graph generator using plotly.

    Parameters:
        df (pd.DataFrame)
        X (str): x-axis column
        Y (str): y-axis column OR "count"
        chart_type (str): bar, line, scatter, stacked_bar, pie
        agg_fn (str): count, mean, sum, max, min
    """

    #aggregetion logic
    if agg_fn == "count":
        grouped = df.groupby(X).size().reset_index(name="value")
        y_col = "value"

    else:
        grouped = df.groupby(X)[Y].agg(agg_fn).reset_index()
        y_col = Y

    #plot type
    if chart_type == "bar":
        fig = px.bar(grouped, x=X, y=y_col)
    elif chart_type == "stacked_bar":
        fig = px.bar(df, x=X, color=Y, barmode="stack")
    elif chart_type == "line":
        fig = px.line(grouped, x=X, y=y_col, markers=True)
    elif chart_type == "scatter":
        fig = px.scatter(grouped, x=X, y=y_col)
    elif chart_type == "pie":
        fig = px.pie(grouped, names=X, values=y_col)
    else:
        raise ValueError(f"Unsupported chart type: {chart_type}")

    fig.update_layout(
        title=f"{agg_fn.capitalize()} of {Y} by {X}",
        xaxis_title=X,
        yaxis_title=y_col,
        template="plotly_white"
    )

    return fig


# Presets for dropdown (could add more)
presets = {

    "Please Select a Preset": {},

    "Launches per Organization": {
        "X": "organisation", "Y": "count", "agg": "count", "chart": "bar"
    },

    "Launches per Country": {
        "X": "location", "Y": "count", "agg": "count", "chart": "bar"
    },

    "Launches per Year": {
        "X": "launch_year", "Y": "count", "agg": "count", "chart": "line"
    },

    "Success Rate per Organization": {
        "X": "organisation", "Y": "success_flag", "agg": "mean", "chart": "bar"
    },

    "Success Rate per Country": {
        "X": "location", "Y": "success_flag", "agg": "mean", "chart": "bar"
    },

    "Price over Time (Year)": {
        "X": "launch_year", "Y": "price", "agg": "mean", "chart": "line"
    },

    "Average Mission Price by Organization": {
        "X": "organisation", "Y": "price", "agg": "mean", "chart": "bar"
    },

    "Total Cost per Organization": {
        "X": "organisation", "Y": "price", "agg": "sum", "chart": "bar"
    },
}


#function that makes downdown and graphs in jupyter notebook files
def dashboard_ui(df):
    preset_dropdown = widgets.Dropdown(
        options=list(presets.keys()),
        description="Graph Type:"
    )

    output = widgets.Output()
    def update_graph(change):
        with output:
            output.clear_output()

            choice = presets[preset_dropdown.value]
            X = choice["X"]
            Y = choice["Y"]
            agg = choice["agg"]
            chart = choice["chart"]

            if chart == "box":
                fig = px.box(df, x=X, y=Y)
            elif chart == "stacked_bar":
                fig = px.bar(df, x=X, color=Y, barmode="stack")
            else:
                fig = template_plot(df, X, Y, chart_type=chart, agg_fn=agg)

            fig.show()

    preset_dropdown.observe(update_graph, names="value")

    display(preset_dropdown, output)



# Run Function
def run_dashboard(path_to_csv):
    df = load_and_prepare(path_to_csv)
    print("Dataset loaded successfully\n")
    dashboard_ui(df)


**Running the below function call will produce the dropdown and graphs that are able to be cycled through. Make sure to change the path given to the function to be accurate. This path should lead to wherever your mission_launches.csv is downloaded.**

In [26]:
### CHANGE THIS PATH AS NEEDED OR THIS WILL FAIL ###

run_dashboard(r"C:\Users\plumm\OneDrive\Documents\Classes\4th Year\B Term 2025\Data Science 3\final github\2025B_WPI_DS3010_Final_Project_Group10\mission_launches.csv")

Dataset loaded successfully



Dropdown(description='Graph Type:', options=('Please Select a Preset', 'Launches per Organization', 'Launches …

Output()